In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: Submit a HyperParameter tuning training job with TensorFlow


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The Vertex AI API service endpoint for prediction.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## Prepare a trainer script

### Package assembly

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Hyperparameter Tuning - Boston Housing\n\
Version: 0.0.0\n\
Summary: Demonstration hyperparameter tuning script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@gmail.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py contents

In [ ]:
%%writefile custom/trainer/task.py
# hyperparameter tuningfor Boston Housing
  
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

def make_dataset():
  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)
    
  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)


### Store training script on your Cloud Storage bucket

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/hpt_boston_housing.tar.gz

## Train a model

### [projects.locations.hyperparameterTuningJob.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/create)

#### Request

In [ ]:
JOB_NAME = "hyperparameter_tuning_" + TIMESTAMP

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4", "accelerator_count": 0},
        "python_package_spec": {
            "executor_image_uri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
            "package_uris": ["gs://" + BUCKET_NAME + "/hpt_boston_housing.tar.gz"],
            "python_module": "trainer.task",
            "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
        },
    }
]

STUDY_SPEC = {
    "metrics": [
        {"metric_id": "val_loss", "goal": aip.StudySpec.MetricSpec.GoalType.MINIMIZE}
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
        {
            "parameter_id": "units",
            "integer_value_spec": {"min_value": 32, "max_value": 256},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
    ],
    "algorithm": aip.StudySpec.Algorithm.RANDOM_SEARCH,
}

hyperparameter_tuning_job = aip.HyperparameterTuningJob(
    display_name=JOB_NAME,
    trial_job_spec={"worker_pool_specs": WORKER_POOL_SPEC},
    study_spec=STUDY_SPEC,
    max_trial_count=6,
    parallel_trial_count=1,
)

print(
    MessageToJson(
        aip.CreateHyperparameterTuningJobRequest(
            parent=PARENT, hyperparameter_tuning_job=hyperparameter_tuning_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "hyperparameterTuningJob": {
    "displayName": "hyperparameter_tuning_20210226020029",
    "studySpec": {
      "metrics": [
        {
          "metricId": "val_loss",
          "goal": "MINIMIZE"
        }
      ],
      "parameters": [
        {
          "parameterId": "lr",
          "discreteValueSpec": {
            "values": [
              0.001,
              0.01,
              0.1
            ]
          },
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterId": "units",
          "integerValueSpec": {
            "minValue": "32",
            "maxValue": "256"
          },
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "algorithm": "RANDOM_SEARCH"
    },
    "maxTrialCount": 6,
    "parallelTrialCount": 1,
    "trialJobSpec": {
      "workerPoolSpecs": [
        {
          "machineSpec": {
            "machineType": "n1-standard-4"
          },
          "replicaCount": "1",
          "pythonPackageSpec": {
            "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
            "packageUris": [
              "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
            ],
            "pythonModule": "trainer.task",
            "args": [
              "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
            ]
          }
        }
      ]
    }
  }
}
```


#### Call

In [ ]:
request = clients["job"].create_hyperparameter_tuning_job(
    parent=PARENT, hyperparameter_tuning_job=hyperparameter_tuning_job
)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/hyperparameterTuningJobs/5264408897233354752",
  "displayName": "hyperparameter_tuning_20210226020029",
  "studySpec": {
    "metrics": [
      {
        "metricId": "val_loss",
        "goal": "MINIMIZE"
      }
    ],
    "parameters": [
      {
        "parameterId": "lr",
        "discreteValueSpec": {
          "values": [
            0.001,
            0.01,
            0.1
          ]
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      },
      {
        "parameterId": "units",
        "integerValueSpec": {
          "minValue": "32",
          "maxValue": "256"
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      }
    ],
    "algorithm": "RANDOM_SEARCH"
  },
  "maxTrialCount": 6,
  "parallelTrialCount": 1,
  "trialJobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-26T02:02:02.787187Z",
  "updateTime": "2021-02-26T02:02:02.787187Z"
}
```


In [ ]:
# The full unique ID for the hyperparameter tuningjob
hyperparameter_tuning_id = request.name
# The short numeric ID for the hyperparameter tuningjob
hyperparameter_tuning_short_id = hyperparameter_tuning_id.split("/")[-1]

print(hyperparameter_tuning_id)

### [projects.locations.hyperparameterTuningJob.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/get)

#### Call

In [ ]:
request = clients["job"].get_hyperparameter_tuning_job(name=hyperparameter_tuning_id)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/hyperparameterTuningJobs/5264408897233354752",
  "displayName": "hyperparameter_tuning_20210226020029",
  "studySpec": {
    "metrics": [
      {
        "metricId": "val_loss",
        "goal": "MINIMIZE"
      }
    ],
    "parameters": [
      {
        "parameterId": "lr",
        "discreteValueSpec": {
          "values": [
            0.001,
            0.01,
            0.1
          ]
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      },
      {
        "parameterId": "units",
        "integerValueSpec": {
          "minValue": "32",
          "maxValue": "256"
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      }
    ],
    "algorithm": "RANDOM_SEARCH"
  },
  "maxTrialCount": 6,
  "parallelTrialCount": 1,
  "trialJobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-26T02:02:02.787187Z",
  "updateTime": "2021-02-26T02:02:02.787187Z"
}
```


## Wait for the study to complete

In [ ]:
while True:
    response = clients["job"].get_hyperparameter_tuning_job(
        name=hyperparameter_tuning_id
    )
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Study trials have not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        print("Study trials have completed:", response.end_time - response.start_time)
        break
    time.sleep(20)

## Review the results of the study

In [ ]:
best = (None, None, None, 0.0)
response = clients["job"].get_hyperparameter_tuning_job(name=hyperparameter_tuning_id)
for trial in response.trials:
    print(MessageToJson(trial.__dict__["_pb"]))
    # Keep track of the best outcome
    try:
        if float(trial.final_measurement.metrics[0].value) > best[3]:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
    except:
        pass

print()
print("ID", best[0])
print("Decay", best[1])
print("Learning Rate", best[2])
print("Validation Accuracy", best[3])

*Example output*:
```
{
  "id": "1",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 80.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 46.61515110294993
      }
    ]
  },
  "startTime": "2021-02-26T02:05:16.935353384Z",
  "endTime": "2021-02-26T02:12:44Z"
}
{
  "id": "2",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 45.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 32.55313952376203
      }
    ]
  },
  "startTime": "2021-02-26T02:15:31.357856840Z",
  "endTime": "2021-02-26T02:24:18Z"
}
{
  "id": "3",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 70.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 42.709188321741614
      }
    ]
  },
  "startTime": "2021-02-26T02:26:40.704476222Z",
  "endTime": "2021-02-26T02:34:21Z"
}
{
  "id": "4",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 173.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "17",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 46.12480219399057
      }
    ]
  },
  "startTime": "2021-02-26T02:37:45.275581053Z",
  "endTime": "2021-02-26T02:51:07Z"
}
{
  "id": "5",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 223.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 24.875632611716664
      }
    ]
  },
  "startTime": "2021-02-26T02:53:32.612612421Z",
  "endTime": "2021-02-26T02:54:19Z"
}
{
  "id": "6",
  "state": "SUCCEEDED",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 123.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "13",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 43.352300690441595
      }
    ]
  },
  "startTime": "2021-02-26T02:56:47.323707459Z",
  "endTime": "2021-02-26T03:03:49Z"
}

ID 1
Decay 0.1
Learning Rate 80.0
Validation Accuracy 46.61515110294993
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_hpt_job = True
delete_bucket = True

# Delete the hyperparameter tuningusing the Vertex AI fully qualified identifier for the custome training
try:
    if delete_hpt_job:
        clients["job"].delete_hyperparameter_tuning_job(name=hyperparameter_tuning_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME